In [1]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
from tweepy import Stream                   
from tweepy.streaming import StreamListener
import json
import csv
import os
from os import path
import got3

In [2]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) | (\w+:\/\/\S+)", " ", tweet).split())

In [3]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) | (\w+:\/\/\S+)", " ", tweet).split())

In [4]:
def get_tweet_sentiment(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'

In [5]:
def write_to_csv(tweet):
    #fields = ["timestamp", "tweet", "user", "verified", "url"]
    fields = ["Date", "Positve", "Negative", "Neutral",]
    filename = "/home/julian/ml-cmpe257/venv/machine-learning-stock-market/TwitterData/AmazonSentiment.csv"
    fileState = path.isfile(filename)
    with open(filename,'a') as fd:        
        writer = csv.writer(fd)
        if(not fileState):
            writer.writerow(fields)
        else:
            print(tweet)
            writer.writerow([tweet.Date, tweet.Positive, tweet.Negative, tweet.Neutral])
 

In [17]:
import pandas as pd
 
tweets_data_path = "/home/julian/ml-cmpe257/venv/machine-learning-stock-market/TwitterData/Amazon.csv"
pdData = pd.read_csv(tweets_data_path, delimiter=",")

all_tweets = []
count = 0
pdData['Date'] = pd.to_datetime(pdData['Date']).dt.date
#list(pdData.groupby('Date')['Tweet'])
pdata = pdData.groupby('Date')['Tweet']
#print (pdata.head())
for date, twt in pdata:
    print (date)
    tweets_day = {}
    tweets_data = []
    tweets_day['Date'] = date
    for t in twt:
        parsedTweet = {}
        #parsedTweet['text'] = t
        parsedTweet['sentiment'] = get_tweet_sentiment(t)
        tweets_data.append(parsedTweet)
    #tweets_day['data'] = tweets_data
    #print (tweets_day)
    
    ptweets = [tweet for tweet in tweets_data if tweet['sentiment'] == 'positive']
    tweets_day['Positive'] = 100*len(ptweets)/len(tweets_data)
    ntweets = [tweet for tweet in tweets_data if tweet['sentiment'] == 'negative']
    tweets_day['Negative'] = 100*len(ntweets)/len(tweets_data)
    tweets_day['Neutral'] = 100*(len(tweets_data) - len(ntweets) - len(ptweets))/len(tweets_data)
    all_tweets.append(tweets_day)
print(all_tweets.head())

2018-09-30
2018-10-01
2018-10-02
2018-10-03
2018-10-04
2018-10-05
2018-10-06
2018-10-07
2018-10-08
2018-10-09
[{'Date': datetime.date(2018, 9, 30), 'Positive': 23.972602739726028, 'Negative': 9.246575342465754, 'Neutral': 66.78082191780823}, {'Date': datetime.date(2018, 10, 1), 'Positive': 31.662269129287598, 'Negative': 10.378188214599824, 'Neutral': 57.959542656112575}, {'Date': datetime.date(2018, 10, 2), 'Positive': 39.095477386934675, 'Negative': 11.557788944723619, 'Neutral': 49.346733668341706}, {'Date': datetime.date(2018, 10, 3), 'Positive': 34.243964421855146, 'Negative': 13.341804320203304, 'Neutral': 52.41423125794155}, {'Date': datetime.date(2018, 10, 4), 'Positive': 31.970649895178198, 'Negative': 16.037735849056602, 'Neutral': 51.9916142557652}, {'Date': datetime.date(2018, 10, 5), 'Positive': 35.93103448275862, 'Negative': 15.655172413793103, 'Neutral': 48.41379310344828}, {'Date': datetime.date(2018, 10, 6), 'Positive': 30.906148867313917, 'Negative': 11.16504854368932

In [18]:
  #fields = ["timestamp", "tweet", "user", "verified", "url"]
fields = ["Date", "Positve", "Negative", "Neutral"]
filename = "/home/julian/ml-cmpe257/venv/machine-learning-stock-market/TwitterData/AmazonSentiment.csv"
fileState = path.isfile(filename)
with open(filename,'a') as fd:        
    writer = csv.writer(fd)
    if(not fileState):
        writer.writerow(fields)
    for sent in all_tweets:
        #tweet = all_tweets[idx]
        print(sent['Date'])
        writer.writerow([sent['Date'], sent['Positive'], sent['Negative'], sent['Neutral']])

2018-09-30
2018-10-01
2018-10-02
2018-10-03
2018-10-04
2018-10-05
2018-10-06
2018-10-07
2018-10-08
2018-10-09
